## Topic - Analyzing Corona Virus

Let's start by getting the complete genome of Coronavirus. Source: https://www.ncbi.nlm.nih.gov/nuccore/NC_045512

> **Basic Information:** Coronavirus is a single stranded RNA-virus (DNA is double stranded). RNA polymers are made up of nucleotides. These nucleotides have three parts: 1) a five carbon Ribose sugar, 2) a phosphate molecule and 3) one of four nitrogenous bases: adenine(a), guanine(g), cytosine(c) or uracil(u) / thymine(t). 

<img src="./images/parts-of-nucleotide.jpg" width="480">

> Thymine is found in DNA and Uracil in RNA. But for following analysis, you can consider (u) and (t) to be analogous.

In [2]:
with open('sars_cov2_data/genome/sars_cov2_genome.txt', 'r') as file:
    sars_cov2 = file.read()

# print(sars_cov2)

Now we only want {a, c, t, g} in sars_cov2.  We want to remove the spaces and numbers. Let's do that:

In [3]:
for s in "\n01234567789 ":
    sars_cov2 = sars_cov2.replace(s, "")

# sars_cov2

In [4]:
len(sars_cov2)

29903

### Q. 1 - What is the 'kolmogorov complexity' of the Coronavirus? 

This question is simply asking - how many bytes of information does it contain. 

In [5]:
import zlib
len(zlib.compress(sars_cov2.encode("utf-8")))

8858

The above result means - The RNA of Coronavirus can contain '8858' bytes of information. This is just an upper-bound. This means - Coronavirus cannot contain more than '8858' bytes of information. Let's see if we can compress it a little more.

In [6]:
import lzma
lzc = lzma.compress(sars_cov2.encode("utf-8"))
len(lzc)

8408

This is a better compression. So we can conclude that - Coronavirus approximately contains 8.3 kB of information. 

### Q. 2 - What type of information does this genome contain? How we can extract it?

The genome contains the information about the proteins it can make. These proteins determine the characteristics of the cell in which they are produced. So we need to extract information about the proteins. To extract this info, we must know - how proteins are formed from the genetic material i.e. DNA/RNA.

> **Basic Information:** RNAs and DNAs form proteins. This is how proteins are formed from DNA. In DNA, A-T/U and G-C form pairs. This pair formation is because - the chemical structure of A, T/U, G and C is such that - A and T are attracted towards each other by 2 hydrogen bonds and G and C together are attracted by 3 hydrogen bonds. A-C and G-T can't form such stable bonds. 

<img src="./images/AT-GC.jpg" width="480">

> What happens during protein formation is:

<img src="./images/transcript-translate-cell.jpg">

> An enzyme called 'RNA polymerase' breaks these hydrogen bonds for a small part, takes one strand of DNA and forms its corresponding paired RNA. This process happens inside the nucleus of the cell. We call this RNA generated as 'mRNA' or 'messenger RNA' because this RNA will come out of nucleus and act like a messaage to Ribosome which will generate proteins accordingly. This process of generation of mRNA is called - **Transcription.** Now Ribosome will read the mRNA in sets of 3 bases. This set of 3 bases is called codon. Codons decide the Amino acids. Depending on the codon read by Ribosome, tRNA (transfer-RNA) brings the appropiate amino acid. These amino acids are then linked using peptide bonds to form a chain called *Polypeptide chain*. At the other end of Ribosome, tRNA is free and can go to take another amino acid. 

> *Note:* Amino acids are organic compounds that contain amine (-NH2) and carboxyl (-COOH) functional groups. There are 20 standard amino acids and 2 non-standard. Of the 20 standard amino acids, nine (His, Ile, Leu, Lys, Met, Phe, Thr, Trp and Val) are called essential amino acids because the human body cannot synthesize them from other compounds at the level needed for normal growth, so they must be obtained from food. Here is the table of codons and their corresponding Amino acids. 'Met' is usually the starting amino acid i.e. 'AUG' forms the start of mRNA. Hence 'AUG' is called *start codon.* 'UAA', 'UGA' and 'UAG' are *stop codons* as they mark the ending of the polypeptide chain, so that a new chain should start from the next codon. 

<img src="./images/genetic-code-table.jpg" width="600">

> This process of generation of chains of amino acids is called - **Translation.** A very long chain of amino acids is called *Protein.* In summary, we can understand the process as:

<img src="./images/transcription-translation.png" width="600">


Now since in Coronavirus, we only has RNA, the process of Transcription won't occur and only Translation will happen. So what we now need to write is - *a translation function*, which takes corona's genome as input and gives back all the polypeptide chains that could be formed from that genome. For that, we first need a dictionary of codons. Following codons' string is copied from 'Genetic code' - Wikipedia.

In [7]:
# Asn or Asp / B	AAU, AAC; GAU, GAC
# Gln or Glu / Z	CAA, CAG; GAA, GAG
# START	AUG

codons = """Ala / A	GCU, GCC, GCA, GCG
Ile / I	AUU, AUC, AUA
Arg / R	CGU, CGC, CGA, CGG; AGA, AGG, AGR;
Leu / L	CUU, CUC, CUA, CUG; UUA, UUG, UUR;
Asn / N	AAU, AAC
Lys / K	AAA, AAG
Asp / D	GAU, GAC
Met / M	AUG
Phe / F	UUU, UUC
Cys / C	UGU, UGC
Pro / P	CCU, CCC, CCA, CCG
Gln / Q	CAA, CAG
Ser / S	UCU, UCC, UCA, UCG; AGU, AGC;
Glu / E	GAA, GAG
Thr / T	ACU, ACC, ACA, ACG
Trp / W	UGG
Gly / G	GGU, GGC, GGA, GGG
Tyr / Y	UAU, UAC
His / H	CAU, CAC
Val / V	GUU, GUC, GUA, GUG
STOP	UAA, UGA, UAG""".strip()

for t in codons.split('\n'):
    t.split('\t')

In [8]:
dec = {}  # Decoder dictionary

for t in codons.split('\n'):
    k, v = t.split('\t')
    if '/' in k:
        k = k.split('/')[-1].strip()
    k = k.replace("STOP", "*")
    v = v.replace(",", "").replace(";", "").lower().replace("u", "t").split(" ")
    for vv in v:
        if vv in dec:
            print("duplicate", vv)
        dec[vv] = k

# dec

In [9]:
len(set(dec.values()))  # We have 21 amino acids in our decoder

21

Now, decoding the genome can result in one of the three possible ways. These 3 ways are called 'reading frames'. 

<img src="./images/reading-frames.png" width="480">

In [10]:
def translation(x, isProtein = False):
    aa = []
    for i in range(0, len(x)-2, 3):
        aa.append(dec[x[i:i+3]])
    aa = ''.join(aa)
    if isProtein:
        if aa[0] != "M" or aa[-1] != "*":
            print("BAD PROTEIN!")
            return None
        aa = aa[:-1]
    return aa

aa = translation(sars_cov2[0:]) + translation(sars_cov2[1:]) + translation(sars_cov2[2:])

In [11]:
polypeptides = aa.split("*")
# polypeptides

In [12]:
len(polypeptides)

1777

In [13]:
long_polypep_chains = list(filter(lambda x: len(x) > 100, aa.split("*")))
# long_polypep_chains

In [14]:
len(long_polypep_chains)

10

This is the genome organisation of Sars-Cov-2. _(Genome organisation is the linear order of genetic material (DNA/RNA) and its division into segments performing some specific function.)_ 

> Note: ORF stands for 'Open Reading Frame', the reading frame in which protein starts with M and ends with *.

Source: https://en.wikipedia.org/wiki/Severe_acute_respiratory_syndrome_coronavirus_2#Phylogenetics_and_taxonomy

<img src="./images/SARS-CoV-2-genome.png" width="900">

Let's see if we can extract all the segments as mentioned here. We will refer to the following source again. Source: https://www.ncbi.nlm.nih.gov/nuccore/NC_045512

Also, if you will see the following genome organisation of Sars-Cov (old coronavirus), you will notice - the structure is very similar to Sars-CoV-2. _(Ignore the detailing given in the structure.)_

<img src="./images/SARS-CoV-1-genome.png" width="800">

In [15]:
# https://www.ncbi.nlm.nih.gov/protein/1802476803 -  
# Orf1a polyprotein, found in Sars-Cov-2 (new Covid 19)
orf1a_v2 = translation(sars_cov2[265:13483], True)

# orf1a_v2

In [16]:
# https://www.uniprot.org/uniprot/A7J8L3
# Orf1a polyprotein, found in Sars-Cov
with open('sars_cov_data/proteins/orf1a.txt', 'r') as file:
    orf1a_v1 = file.read().replace('\n', '')

# orf1a_v1

In [17]:
len(orf1a_v1), len(orf1a_v2)

(4382, 4405)

Usually orf1b is not studied alone but along with orf1a. So we need to look at 'orf1ab'. But just to prove that the length of orf1b is 2595, here is just finding the length of orf1b in SARS-CoV-2.

In [18]:
# For orf1b_v1, refer - https://www.uniprot.org/uniprot/A0A0A0QGJ0
orf1b_v2 = translation(sars_cov2[13467:21555])

# Length calculated from first 'M'. The last base is *, so extra -1 for that. 
len(orf1b_v2) - orf1b_v2.find('M') - 1   

2595

In [19]:
# https://www.ncbi.nlm.nih.gov/protein/1796318597 - 
# Orf1ab polyprotein - found in Sars-cov-2
orf1ab_v2 = translation(sars_cov2[265:13468]) + translation(sars_cov2[13467:21555])

In [20]:
# https://www.uniprot.org/uniprot/A7J8L2
# Orf1ab polyprotein - found in Sars-cov

with open('sars_cov_data/proteins/orf1ab.txt', 'r') as file:
    orf1ab_v1 = file.read().replace('\n', '')

In [21]:
len(orf1ab_v2), len(orf1ab_v1)

(7097, 7073)

So by now, we have extracted Orf1a and Orf1b RNA segments. 

In [22]:
# https://www.ncbi.nlm.nih.gov/protein/1796318598
# Spike glycoprotein - found in Sars-cov-2

spike_v2 = translation(sars_cov2[21562:25384], True)

In [23]:
# https://www.uniprot.org/uniprot/P59594
# Spike glycoprotein - found in Sars-cov

with open('sars_cov_data/proteins/spike.txt', 'r') as file:
    spike_v1 = file.read().replace('\n', '')

In [24]:
len(spike_v2), len(spike_v1)

(1273, 1255)

In [25]:
# https://www.ncbi.nlm.nih.gov/gene/43740569
# orf3a protein found in Sars-cov-2.

orf3a_v2 = translation(sars_cov2[25392:26220], True)

In [26]:
# https://www.uniprot.org/uniprot/J9TEM7

with open('sars_cov_data/proteins/orf3a.txt', 'r') as file:
    orf3a_v1 = file.read().replace('\n', '')

In [27]:
len(orf3a_v2), len(orf3a_v1)

(275, 274)

By now, you must have seen that there is a very little change in the corresponding protein lengths of SARS-CoV and SARS-CoV-2. So, **Can we say - there isn't much difference between the proteins of two viruses?** And the answer is - **NO!** 

This is because - length of the proteins is not the accurate measure of how dissimilar they are. So now we have another question.

### Q. 3 How much different is the protein of this novel coronavirus as compared to the older one?

The answer is - **The Edit Distance.** In computational linguistics and computer science, edit distance is a way of quantifying how dissimilar two strings (e.g., words) are to one another by counting the minimum number of operations required to transform one string into the other. In bioinformatics, it can be used to quantify the similarity of DNA sequences, which can be viewed as strings of the letters A, C, G and T. 

Source: https://en.wikipedia.org/wiki/Edit_distance

Let's calculate the edit distance of the genomes of the two versions of coronaviruses. 

Source of complete genome of old coronavirus: https://www.ncbi.nlm.nih.gov/nuccore/30271926

In [28]:
with open('sars_cov_data/genome/sars_cov_genome.txt', 'r') as file:
    sars_cov = file.read()

# print(sars_cov)

In [29]:
for s in "\n01234567789 ":
    sars_cov = sars_cov.replace(s, "")

In [30]:
import lzma
lzc_v1 = lzma.compress(sars_cov.encode("utf-8"))
len(lzc_v1)

8412

In [31]:
len(lzc_v1) - len(lzc)

4

By looking at the kolmogorov complexity of the old coronavirus, you can comment that both coronaviruses differ by about 4 bytes of information. But here is the reality:

(Since my PC hangs in executing the following command, I am posting the screenshot of it which I executed on google colab.)

![](./images/edit-distance.png)

And here is what happens when you compare the lengths:

In [32]:
len(sars_cov2) - len(sars_cov)

152

From this, we can see that - Novel coronavirus differ alot than expected from old coronavirus. Now that we know - the difference between two DNAs/RNAs is measured by calculating edit-distance, we can now just simply complete extracting other proteins. 

In [33]:
# https://www.ncbi.nlm.nih.gov/gene/43740570 - Envelope protein in Cov-2
envelope_v2 = translation(sars_cov2[26244:26472], True)

In [34]:
len(envelope_v2)

75

In [35]:
# https://www.ncbi.nlm.nih.gov/gene/43740571 - Membrane Glycoprotein in Cov-2
membrane_v2 = translation(sars_cov2[26522:27191], True)

In [36]:
len(membrane_v2)

222

In [37]:
# https://www.ncbi.nlm.nih.gov/gene/43740572 - Orf6 in Cov-2
orf6_v2 = translation(sars_cov2[27201:27387], True)

In [38]:
len(orf6_v2)

61

In [39]:
# https://www.ncbi.nlm.nih.gov/gene/43740573 - orf7a in Cov-2
orf7a = translation(sars_cov2[27393:27759], True)

In [40]:
len(orf7a)

121

In [41]:
# https://www.ncbi.nlm.nih.gov/gene/43740574 - orf7b in Cov-2
orf7b = translation(sars_cov2[27755:27887], True)

In [42]:
len(orf7b)

43

In [43]:
# https://www.ncbi.nlm.nih.gov/gene/43740577 - orf8 in Cov-2
orf8 = translation(sars_cov2[27893:28259], True)

In [44]:
len(orf8)

121

In [45]:
# https://www.ncbi.nlm.nih.gov/gene/43740576 - orf10 in Cov-2
orf10 = translation(sars_cov2[29557:29674], True)

In [46]:
len(orf10)

38

In [53]:
# Run following command on terminal (without quotes):
# !pip install nglview
# "jupyter-nbextension enable nglview --py --sys-prefix"

import nglview as nv
view = nv.show_pdbid("3pqr")
view

# Visualizing 

NGLWidget()